---
# Name of Method - Model 3
---
In this notebook, ...


## Results

---

### Importing necessary library

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Read data file

In [5]:
churn_df = pd.read_excel('../data/churn_cleaned_featEng.xlsx')
churn_df

,Tenure Months,Monthly Charges,Churn Value,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Internet Service_Fiber optic,Internet Service_No,Online Security_Yes,Online Backup_Yes,Device Protection_Yes,Tech Support_Yes,Contract_One year,Contract_Two year,Paperless Billing_Yes,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,2,53.85,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1
1,2,70.70,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0
2,8,99.65,1,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0
3,28,104.80,1,0,1,1,1,0,0,0,1,1,0,0,1,0,1,0
4,49,103.70,1,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,72,21.15,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
7039,24,84.80,0,0,1,1,0,0,1,0,1,1,1,0,1,0,0,1
7040,72,103.20,0,0,1,1,1,0,0,1,1,0,1,0,1,1,0,0
7041,11,29.60,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0


---

<center>
    
## Preparing data

</center>

---

### Separate X and y features

In [8]:
# Seperate X and y features
X = churn_df.drop(columns=['Churn Value'])
y = churn_df['Churn Value']

### Split dataset (training/testing)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=5)

### Converting data subset to dataframe 

In [12]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### Scale X features

In [14]:
# Create instance of scaler
scaler = StandardScaler()

# Scale the data
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

---

<center>
    
## Model

</center>

---

### Find best parameters for model


In [18]:
import features

# Create instance of feature selector
selection1= features.features
selection2= features.features_chi

selection1best5 = list(dict(sorted(selection1.items(), key=lambda item: item[1], reverse=True)[:5]).keys())
selection1best10 = list(dict(sorted(selection1.items(), key=lambda item: item[1], reverse=True)[:10]).keys())
selection2best5 = list(dict(sorted(selection2.items(), key=lambda item: item[1], reverse=True)[:5]).keys())
selection2best10 = list(dict(sorted(selection2.items(), key=lambda item: item[1], reverse=True)[:10]).keys())

### Run model on training dataset

### Investigate best model's predictive features

In [21]:
from sklearn.linear_model import LogisticRegression

using_features = selection1best5

# Create instance of model
model = LogisticRegression()

# Fit the model
model.fit(X_train[using_features], y_train)

# Predict the model
y_pred = model.predict(X_test[using_features])

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.93      0.87       520
           1       0.68      0.45      0.54       185

    accuracy                           0.80       705
   macro avg       0.75      0.69      0.71       705
weighted avg       0.79      0.80      0.79       705



---

<center>
    
## Validation

</center>

---

### Run model on testing dataset

In [24]:
# Run the mode on testing data
y_pred = model.predict(X_test[using_features])

# Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.93      0.87       520
           1       0.68      0.45      0.54       185

    accuracy                           0.80       705
   macro avg       0.75      0.69      0.71       705
weighted avg       0.79      0.80      0.79       705



### Model's validation

In [26]:
# Models validation
from sklearn.model_selection import cross_val_score

# lets do cross validation, to find the best hyperparameters
# Create instance of model
# we will be doing cross validation to find the best hyperparameters out of the following
# C: Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty: Used to specify the norm used in the penalization. The ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers support only l2 penalties.
# solver: Algorithm to use in the optimization problem.
# tol: Tolerance for stopping criteria.
# max_iter: Maximum number of iterations taken for the solvers to converge.
model = LogisticRegression()

# Create a dictionary of hyperparameters
hyperparameters = {
    'C': [0.001,  0.1, 10, 100, 200,500],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'tol': [0.0001, 0.001, 0.01,0.1],
    'max_iter': [100, 1000, 10000]
}

# Create instance of GridSearchCV
grid = GridSearchCV(model, hyperparameters, cv=5)

# Fit the model
grid.fit(X_train[using_features], y_train)

# Print the best hyperparameters
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_index_)
print(grid.best_estimator_.coef_)

{'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.1}
0.8008827578995067
LogisticRegression(C=10, penalty='l1', solver='liblinear', tol=0.1)
51
[[-0.04000971  1.08892221 -1.08742131  0.51297198 -1.54859543]]


---

<center>
    
## Results

</center>

---

### Results

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.93      0.87       520
           1       0.68      0.45      0.54       185

    accuracy                           0.80       705
   macro avg       0.75      0.69      0.71       705
weighted avg       0.79      0.80      0.79       705

